## TP4(Implementation des KNNs applique au spam et ham)
   ##  GROUPE 3 CONSTITUE DE :
   ##        - LEPAFO MOFFO Baurel               21S2782
   ##        - MANFOUO SELATSA Ridano            16U2848
   ##        - TEINGA KAMGUEN Emmanuel Rufin     21S2822
   ##        - MEBODO ONOMO JUSTIN JORDAN        19M2217

In [12]:
## Utilisation des KNN pour detecter les SPAMs et les HAMs

In [13]:
## importation des utilitaires necessaires
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [14]:
## chargement des donnees
def load_data():
    print("chargement des données...")
    ## on retourne la liste des fichier via os.listdir
    ham_files_location = os.listdir("dataset/ham")
    spam_files_location = os.listdir("dataset/spam")
    data = []
    # chargement de email de classe Ham
    for file_path in ham_files_location:
        f = open("dataset/ham/" + file_path, "r") ## ici on ouvre un fichier de ham et envoi son contenu dans text
        text = str(f.read()) ## ici on converti le contenu du fichier en string
        data.append([text, "ham"])## ici on envoi la valeur du fichier de forme str dans data
    
    # chargement des emails de classes spam
    for file_path in spam_files_location: ## ici on fait de meme que les ham mais pour les spam
        f = open("dataset/spam/" + file_path, "r") 
        text = str(f.read())
        data.append([text, "spam"])
    
    data = np.array(data) ## on tranforme les donnees en un np.array
    
    print("Tache 1: chargement des donnees")
    return data

In [15]:
## maintenant nous devons praitraiter nos donnees

def preprocess_data(data):
    print("Preprocessing data...")
    
    punc = string.punctuation           ##liste des punctuations
    sw = stopwords.words('english')     ##liste des Stopwords
    for record in data:
        ## suppression de ponctuation et symboles standard
        for item in punc:
            record[0] = record[0].replace(item, "")
        
        # on met tous les mots en minuscules et on supprime les stopwords 
        splittedWords = record[0].split() ## split ici est pour obtenir la liste de tous les  emails
        newText = ""
        for word in splittedWords:
            if word not in sw: ## on evite de prendre dans notre liste de mot des symbol non desire
                word = word.lower()
                newText = newText + " " + word  
        record[0] = newText
        
    print("Tache 2: pretraitement des donnees")        
    return data

In [16]:
## Decoupage des donnees en ensembles d'entrainnement et de test
def split_data(data):
    print("Decoupage des donnees...")
    
    features = data[:, 0]   # Tableau des emails
    labels = data[:, 1]     # Tableau des labels
    print(labels)
    training_data, test_data, training_labels, test_labels =\
        train_test_split(features, labels, test_size = 0.27, random_state = 42)
    
    print("tache 3: donnees decoupees")
    return training_data, test_data, training_labels, test_labels

In [17]:
## creation de la fonction comptant le nombre d'occurence d'un mot via l'outil de str split
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1
    
    return wordCounts

In [18]:
## ici on calcule la distance euclidienne entre les observations de l'ensemble de test et ceux de l'ensemble d'Entrainement
def euclidean_difference(test_WordCounts, training_WordCounts):
    total = 0
    for word in test_WordCounts:
        if word in test_WordCounts and word in training_WordCounts:
            total += (test_WordCounts[word] - training_WordCounts[word])**2 ## mise au carre de la difference de distance de mot
            del training_WordCounts[word] ## suppression du mot commun tuiliser dans les 2 ensembles
        else:
            total += test_WordCounts[word]**2 ## pas besoin de calculer la difference vu que le mot est dans un seul endroit
            
    for word in training_WordCounts:
        total += training_WordCounts[word]**2
        
    return total**0.5 ## ici on y fait la racine carre de la valeur totale pour avoir la difference finalle

In [19]:
## retourner la classe du mail correspondant(soit un spam soit un ham selon le nombre)
def get_class(selected_Kvalues):
    spam_count = 0
    ham_count = 0
    
    for value in selected_Kvalues:
        if value[0] == "spam":
            spam_count += 1
        else:
            ham_count += 1
    if spam_count > ham_count:
        return "spam"
    else:
        return "ham"

In [20]:
## classifieur knn

def knn_classifier(training_data, training_labels, test_data, K, tsize):
    print("Lancement du classifier KNN...")
    
    result = []
    counter = 1
    
    # compteur de mots pour entrainement des emails
    
    training_WordCounts = [] 
    for training_text in training_data:
            training_WordCounts.append(get_count(training_text))
    for test_text in test_data:
        
        similarity = [] #liste des similarite via la distance euclidienn
        test_WordCounts = get_count(test_text)  #compteur de mot pour des emails
        
        # retourner la distance euclidienne entre ensemble 
       
        for index in range(len(training_data)):
            euclidean_diff =\
                euclidean_difference(test_WordCounts, training_WordCounts[index])
            similarity.append([training_labels[index], euclidean_diff])
        
        # on tri ici la liste a l'Aide des differences euclidienne
        
        similarity = sorted(similarity, key = lambda i:i[1])
        
        #Selection de K plus proche voisins
        
        selected_Kvalues = [] 
        for i in range(K):
            selected_Kvalues.append(similarity[i])
            
            #Class predite pour les emails
        
        result.append(get_class(selected_Kvalues))
    return result

In [21]:
## mise en oeuvre de notre classifieur
def main(K):
    data = load_data()
    data = preprocess_data(data)
    training_data, test_data, training_labels, test_labels = split_data(data)
    tsize = len(test_data)
    result = knn_classifier(training_data, training_labels, test_data[:tsize], K, tsize) 
    accuracy = accuracy_score(test_labels[:tsize], result)
    print("training data size\t: " + str(len(training_data)))
    print("test data size\t: " + str(len(test_data)))
    print("K value\t: " + str(K))
    print("Samples tested\t: " + str(tsize))
    print("% accuracy\t:" + str(accuracy * 100))
    print("Number correct\t: " + str(int(accuracy * tsize)))
    print("Number wrong\: " + str(int((1 - accuracy) * tsize)))

main(11)

chargement des données...
Tache 1: chargement des donnees
Preprocessing data...
Tache 2: pretraitement des donnees
Decoupage des donnees...
['ham' 'ham' 'ham' ... 'spam' 'spam' 'spam']
tache 3: donnees decoupees
Lancement du classifier KNN...
training data size	: 4275
test data size	: 1582
K value	: 11
Samples tested	: 1582
% accuracy	:76.73830594184577
Number correct	: 1214
Number wrong\: 368
